In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import pandas as pd
from google.colab.patches import cv2_imshow

In [ ]:
root = '...'#training data comes from multiple sources and they are stored in this folder

In [ ]:
imageDirs = [root+'/G1020/Images', root+'/ORIGA/Images',root+'/REFUGE/Images',root+'/PAPILA/Images']
ROIMaskDirs = [root+'/G1020/ROI', root+'/ORIGA/ROI',root+'/REFUGE/ROI', root+'/PAPILA/ROI']
segmentedMaskDirs = [root+'/G1020/Masks', root+'/ORIGA/Masks',root+'/REFUGE/Masks',root+'/PAPILA/Masks']

In [ ]:
for root, dirs, files in os.walk(imageDirs[0]):
    for file in files:
        if file.endswith('.json'):
            os.remove(os.path.join(root, file))

In [ ]:
imagePaths = []
for dir in imageDirs:
    imagePaths.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])

ROIMaskPaths = []
for dir in ROIMaskDirs:
    ROIMaskPaths.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])
    
segmentedMaskPaths = []
for dir in segmentedMaskDirs:
    segmentedMaskPaths.extend([os.path.join(dir, f) for f in sorted(os.listdir(dir))])


In [ ]:
for i in range(len(imagePaths)):
    image = cv2.imread(imagePaths[i])
    ROIMask = cv2.imread(ROIMaskPaths[i], 0)
    mask = cv2.imread(segmentedMaskPaths[i], 0)

    contours, _ = cv2.findContours(ROIMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    x,y,w,h = cv2.boundingRect(contours[0])

    croppedImage = image[y:y+h, x:x+w]
    croppedMask = mask[y:y+h, x:x+w]
    
    newImagePath = imagePaths[i].replace('/Images/', '/CroppedImages/')
    newMaskPath = segmentedMaskPaths[i].replace('/Masks/', '/CroppedMasks/')

    os.makedirs(os.path.dirname(newImagePath), exist_ok=True)
    os.makedirs(os.path.dirname(newMaskPath), exist_ok=True)

    cv2.imwrite(newImagePath, croppedImage)
    cv2.imwrite(newMaskPath , croppedMask)